In [1]:
import cat4py as cat
import numpy as np
import os

In [2]:
class Array(cat.Container):
    
    def __init__(self, dtype, pshape=None, filename=None, **kargs):
        self.dtype = np.dtype(dtype)
        kargs["metalayers"] = {"numpy": {"dtype": str(self.dtype)}}
        self.kargs = kargs
        super(cat.Container, self).__init__(pshape, filename, **kargs)
    
    def iter_write(self):
        return cat.Container.iter_write(self, self.dtype)
    
    def iter_read(self, blockshape):
        return cat.Container.iter_read(self, blockshape, self.dtype)
    
    def copy(self, pshape=None, filename=None, **kargs):
        arr = cat.Container.copy(self, pshape, filename, **kargs)
        arr.__class__ = Array
        arr.dtype = self.dtype
        return arr
    
    def __array__(self):
        return cat.Container.to_numpy(self, self.dtype)

def empty_array(dtype, shape, pshape=None, filename=None, **kargs):
    arr = Array(dtype, pshape, filename, **kargs)
    arr.updateshape(shape)
    return arr

def from_file(filename):
    arr = cat.from_file(filename)
    arr.__class__ = Array
    dtype = arr.get_metalayer("numpy")[b"dtype"]
    arr.dtype = np.dtype(dtype)
    return arr

In [3]:
a = empty_array(np.int32, shape=(32, 32), pshape=(16, 16), filename="notebook.cat")

In [4]:
for block, info in a.iter_write():
    block[:] = np.ones((info.shape), dtype=np.int32)

In [5]:
b = a.copy(pshape=(12, 3), filename="notebook2.cat")

In [6]:
np.matmul(a, b)

array([[32, 32, 32, ..., 32, 32, 32],
       [32, 32, 32, ..., 32, 32, 32],
       [32, 32, 32, ..., 32, 32, 32],
       ...,
       [32, 32, 32, ..., 32, 32, 32],
       [32, 32, 32, ..., 32, 32, 32],
       [32, 32, 32, ..., 32, 32, 32]], dtype=int32)

In [7]:
os.remove("notebook.cat")
os.remove("Notebook2.cat")